In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
import pandas as pd
import os
import random
import warnings
warnings.filterwarnings('ignore')
import lightgbm as lgb
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
import numpy as np
from utils import custom_train_test_split, lgbm_predict, post_slack
base_feats = ['KnowledgeTag', 'user_correct_answer', 'user_total_answer', 
         'user_acc', 'test_mean', 'test_sum', 'tag_mean','tag_sum']
# train과 test 데이터셋은 사용자 별로 묶어서 분리를 해주어야함

## 1. 데이터 로딩
data_dir = '/opt/ml/input/data' # 경로
# train_file_path = os.path.join(data_dir, 'train_data.csv') # 데이터
# test_file_path = os.path.join(data_dir, 'test_data.csv')
after_fe_path = os.path.join(data_dir, 'after_fe_train_test.pkl')
df = pd.read_pickle(after_fe_path)

train_df = df[df.kind=='train']
train, valid = custom_train_test_split(train_df, ratio=0.7, seed=13) # 훈련데이터 split
test = df[df.kind=='test'] # 테스트 데이터
train2 = test[test.answerCode!=-1] # 테스트데이터 마지막 제출 2번쨰꺼까지 훈련데이터로 사용
train = pd.concat([train_df,train2]) # 훈련데이터 병합
train.shape, valid.shape, test.shape

x_train = train.drop('answerCode',axis=1)
y_train = train[['answerCode']]

x_valid = valid.drop('answerCode',axis=1)
y_valid = valid[['answerCode']]
x_train.shape, y_train.shape, x_valid.shape, y_valid.shape

((1845527, 22), (1845527, 1), (2007, 22), (2007, 1))

In [7]:
df[FEATS]

,KnowledgeTag,user_correct_answer,user_total_answer,big_category,mid_category,problem_num,month,day,dayname,hour,user_acc,test_mean,test_sum,test_std,tag_std,tag_mean,tag_sum
0,7224,NaN,0,6,1,1,3,24,0,0,NaN,0.947683,1268,0.222749,0.207410,0.955022,637
1,7225,1.0,1,6,1,2,3,24,0,0,1.000000,0.947683,1268,0.222749,0.281603,0.913187,3040
2,7225,2.0,2,6,1,3,3,24,0,0,1.000000,0.947683,1268,0.222749,0.281603,0.913187,3040
3,7225,3.0,3,6,1,4,3,24,0,0,1.000000,0.947683,1268,0.222749,0.281603,0.913187,3040
4,7225,4.0,4,6,1,5,3,24,0,0,1.000000,0.947683,1268,0.222749,0.281603,0.913187,3040
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260109,8832,8.0,11,4,130,1,10,14,5,23,0.727273,0.604762,127,0.499738,0.499618,0.584848,193
260110,8832,8.0,12,4,130,2,10,14,5,23,0.666667,0.604762,127,0.499738,0.499618,0.584848,193
260111,8244,9.0,13,4,130,3,10,14,5,23,0.692308,0.604762,127,0.499738,0.476214,0.725490,111
260112,8244,10.0,14,4,130,4,10,14,5,23,0.714286,0.604762,127,0.499738,0.476214,0.725490,111


In [2]:
# 사용할 Feature 설정
FEATS = ['KnowledgeTag',
         'user_correct_answer',
         'user_total_answer',
         'big_category',
         'mid_category',
         'problem_num',
         'month','day','dayname','hour',
         'user_acc',
         'test_mean',
         'test_sum',
         'test_std',
         'tag_std',
         'tag_mean',
         'tag_sum',
        ]

params = {
#     "max_depth": 8,  # 8,
#     "min_data_in_leaf": 1000,
    # "feature_fraction": 0.6,  # 0.8,
#     "bagging_fraction": 0.75,
    # "max_cat_group": 64,
    "objective": "binary",
#     "boosting": "gbdt",  # dart
#     "learning_rate": 0.01,  # 0.01,
    # "bagging_freq": 5,
    "seed": 42,
    # "max_bin": 50,
#     "num_leaves": 80,  # 40,
#     "metric": "auc",
}

model_auc = lgb.LGBMClassifier(
    **params,
    n_estimators=3200,
    silent=-1,
)

model_loss = lgb.LGBMClassifier(
    **params,
    n_estimators=3200,
    silent=-1,
)

In [3]:
model_loss.fit(
    X=x_train[FEATS],
    y=y_train,
#     early_stopping_rounds=2000,
    eval_set=[(x_train[FEATS], y_train), (x_valid[FEATS], y_valid)],
    eval_names=["train", "vavclid"],
    eval_metric="logloss",`c
    verbose=100,
)

preds = model_loss.predict_proba(x_valid[FEATS])[:, 1]
acc = accuracy_score(y_valid, np.where(preds >= 0.5, 1, 0))
auc = roc_auc_score(y_valid, preds)
print(f'VALID AUC : {auc} ACC : {acc}\n')

[100]	train's binary_logloss: 0.544973	valid's binary_logloss: 0.628527
[200]	train's binary_logloss: 0.540788	valid's binary_logloss: 0.625436
[300]	train's binary_logloss: 0.537663	valid's binary_logloss: 0.623622
[400]	train's binary_logloss: 0.534668	valid's binary_logloss: 0.621837
[500]	train's binary_logloss: 0.531612	valid's binary_logloss: 0.618392
[600]	train's binary_logloss: 0.528818	valid's binary_logloss: 0.618177
[700]	train's binary_logloss: 0.526298	valid's binary_logloss: 0.617426
[800]	train's binary_logloss: 0.523981	valid's binary_logloss: 0.61737
[900]	train's binary_logloss: 0.522013	valid's binary_logloss: 0.616918
[1000]	train's binary_logloss: 0.520075	valid's binary_logloss: 0.617071
[1100]	train's binary_logloss: 0.518084	valid's binary_logloss: 0.616309
[1200]	train's binary_logloss: 0.515946	valid's binary_logloss: 0.616555
[1300]	train's binary_logloss: 0.514273	valid's binary_logloss: 0.616169
[1400]	train's binary_logloss: 0.51266	valid's binary_logloss

LGBMClassifier(n_estimators=3200, objective='binary', seed=42, silent=-1)

VALID AUC : 0.7197475650964024 ACC : 0.6567015445939213



In [4]:
model_auc.fit(
    X=x_train[FEATS],
    y=y_train,
    early_stopping_rounds=1000,
    eval_set=[(x_train[FEATS], y_train), (x_valid[FEATS], y_valid)],
    eval_names=["train", "valid"],
    eval_metric="auc",
    verbose=100,
)

preds = model_auc.predict_proba(x_valid[FEATS])[:, 1]
acc = accuracy_score(y_valid, np.where(preds >= 0.5, 1, 0))
auc = roc_auc_score(y_valid, preds)
print(f'VALID AUC : {auc} ACC : {acc}\n')

[100]	train's auc: 0.758562	train's binary_logloss: 0.544973	valid's auc: 0.703533	valid's binary_logloss: 0.628527
[200]	train's auc: 0.763481	train's binary_logloss: 0.540788	valid's auc: 0.707249	valid's binary_logloss: 0.625436
[300]	train's auc: 0.76711	train's binary_logloss: 0.537663	valid's auc: 0.709369	valid's binary_logloss: 0.623622
[400]	train's auc: 0.77068	train's binary_logloss: 0.534668	valid's auc: 0.711506	valid's binary_logloss: 0.621837
[500]	train's auc: 0.774298	train's binary_logloss: 0.531612	valid's auc: 0.716893	valid's binary_logloss: 0.618392
[600]	train's auc: 0.777622	train's binary_logloss: 0.528818	valid's auc: 0.71735	valid's binary_logloss: 0.618177
[700]	train's auc: 0.780537	train's binary_logloss: 0.526298	valid's auc: 0.718182	valid's binary_logloss: 0.617426
[800]	train's auc: 0.78319	train's binary_logloss: 0.523981	valid's auc: 0.718269	valid's binary_logloss: 0.61737
[900]	train's auc: 0.785508	train's binary_logloss: 0.522013	valid's auc: 0.7

LGBMClassifier(n_estimators=3200, objective='binary', seed=42, silent=-1)

VALID AUC : 0.7215265354800239 ACC : 0.6631788739412058



In [5]:
post_slack("done")

In [ ]:
lgbm_predict(test, model, FEATS, 'test.csv')